In [1]:
from pathlib import Path
import pandas as pd
import json
import re
import subprocess

# Export whole project to Markdown files for sharing with LLM
- Specs_Todos
- Project_Structure
- Code_Base

In [2]:
# Folders
base_dir = Path('D:\Entwicklung\MetaNode')
specs_dir = base_dir / "specs"
static_dir = base_dir / "static"

In [3]:
# Sources
specs_file = specs_dir / "Specs_20250622.md"
todos_file = specs_dir / "todos.md"
template_structure = static_dir / "template_structure.md"

# Targets
specs_todos = static_dir / "1_Specs_Todos.md"
project_structure = static_dir / "2_Project_Structure.md"
code_base = static_dir / "3_Code_Base.md"
all_in_one = static_dir / "0_All_in_one.md"

In [4]:
# Annotated file list
xl_file = static_dir / 'file_meta.xlsx'
xl_fullname = str(xl_file.resolve())
print(xl_file.exists(), xl_fullname)

True D:\Entwicklung\MetaNode\static\file_meta.xlsx


## Creating "Specs_Todos.md"

In [5]:
text_specs = specs_file.read_text(encoding='utf-8')
text_todos = todos_file.read_text(encoding='utf-8')
text = "## Specifications\n"
text += text_specs + '\n\n'
text += "## Todos\n"
text += text_todos + '\n'
specs_todos.write_text(text, encoding ='utf-8')

5382

## Creating "Project_Structure.md"

In [6]:
df0 = pd.read_excel(xl_fullname).fillna("")
df = df0[df0['include'] == 1]

In [7]:
def get_relative_dir(filepath, base_dir):
    try:
        return Path(filepath).parent.relative_to(base_dir)
    except ValueError:
        return Path(filepath).parent  # ggf. .resolve() hinzufügen

In [8]:
df.loc[:, 'dir'] = df['filepath'].apply(lambda fp: str(Path(fp).parent))

C:\Users\gay5vi\AppData\Local\Temp\ipykernel_7740\1739340866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'dir'] = df['filepath'].apply(lambda fp: str(Path(fp).parent))


In [9]:
df.sample(3)
# df

,filepath,include,purpose,status,comment,dir
4,core/undo_manager.py,1,Undo manager,,,core
13,models/node_model.py,1,,,,models
30,schemas/chapter_meta.json,1,,,,schemas


In [10]:
dirs = df['dir'].astype('str').unique()
dirs

array(['.', 'core', 'models', 'resources', 'schemas', 'ui', 'widgets'],
      dtype=object)

In [11]:
def get_files_report(d):
    files = df[df['dir'] == d][['filepath', 'purpose', 'status', 'comment']]
    files_list = []
    for _,row in files.iterrows():
        dict_ = row.to_dict()
        text_list = []
        for k,v in row.items():
            if v:
                t = k + ': ' + '"' + str(v) + '"'
                text_list.append(t)
        text = '{' + ', '.join(text_list) + '}'
        files_list.append('- ' + text)
    files_text = '\n'.join(files_list)
    return files_text

In [12]:
print(get_files_report('ui'))

- {filepath: "ui/content_panel_view.py"}
- {filepath: "ui/main_window.py", purpose: "Main module, called by main."}
- {filepath: "ui/node_editor_panel.py"}
- {filepath: "ui/tree_view.py"}


In [13]:
text = template_structure.read_text(encoding='utf-8')

for dir_ in dirs:
    to_insert = get_files_report(dir_)
    to_insert = '**Dateien:**\n' + to_insert
    pattern = re.compile(rf"\{{{dir_}\}}")
    text = pattern.sub(to_insert, text)
project_structure.write_text(text, encoding='utf-8')

2175

## Creating "Code_Base.md"

In [14]:
text = ''
for _,row in df.iterrows():
    filepath = row['filepath']
    file = base_dir / filepath
    file_text = file.read_text(encoding='utf-8')
    text += '**' + filepath + ':**\n\n' + file_text + '---\n\n'
code_base.write_text(text, encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Entwicklung\\MetaNode\\widgets\\content_panel_view.py'

## Creating "All_in_one.md"

In [ ]:
text = ''
for file in [specs_todos, project_structure, code_base]:
    text += file.read_text(encoding='utf-8') + '---\n\n'
all_in_one.write_text(text, encoding='utf-8')

70078